# Ensemble Learning

## Initial Imports

In [24]:
import warnings
warnings.filterwarnings('ignore')

In [25]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [26]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [27]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [28]:
# Create our features
X = df.copy() # YOUR CODE HERE
X.drop("loan_status", axis=1, inplace=True)
X.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,n,27.24,0.0,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,n,20.23,0.0,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,n,24.26,0.0,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,n,31.44,0.0,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,n,18.76,0.0,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [29]:
X = df.copy() # YOUR CODE HERE
X.drop("home_ownership", axis=1, inplace=True)
X.head()#Dropping any extra data that doesn't have a numeric data

,loan_amnt,int_rate,installment,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,0.0,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,105000.0,Verified,Mar-2019,low_risk,n,20.23,0.0,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,56000.0,Verified,Mar-2019,low_risk,n,24.26,0.0,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,92000.0,Verified,Mar-2019,low_risk,n,31.44,0.0,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,0.0,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [30]:
X = df.copy() # YOUR CODE HERE
X.drop("verification_status", axis=1, inplace=True)
X.head() #Dropping any estra data that doesn't have a numeric data

,loan_amnt,int_rate,installment,home_ownership,annual_inc,issue_d,loan_status,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Mar-2019,low_risk,n,27.24,0.0,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Mar-2019,low_risk,n,20.23,0.0,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Mar-2019,low_risk,n,24.26,0.0,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Mar-2019,low_risk,n,31.44,0.0,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Mar-2019,low_risk,n,18.76,0.0,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [31]:
print(df.columns) #making sure no data w/o a numeric data is still present

Index(['loan_amnt', 'int_rate', 'installment', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'dti',
       'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal',
       'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_amnt', 'next_pymnt_d', 'collections_12_mths_ex_med',
       'policy_code', 'application_type', 'acc_now_delinq', 'tot_coll_amt',
       'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m',
       'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util',
       'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util',
       'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_si

In [32]:
print(df) #second check

       loan_amnt  int_rate  installment home_ownership  annual_inc  \
0        10500.0    0.1719       375.35           RENT     66000.0   
1        25000.0    0.2000       929.09       MORTGAGE    105000.0   
2        20000.0    0.2000       529.88       MORTGAGE     56000.0   
3        10000.0    0.1640       353.55           RENT     92000.0   
4        22000.0    0.1474       520.39       MORTGAGE     52000.0   
...          ...       ...          ...            ...         ...   
68812    10000.0    0.1502       346.76           RENT     26000.0   
68813    12000.0    0.2727       368.37           RENT     63000.0   
68814     5000.0    0.1992       185.62       MORTGAGE     52000.0   
68815    40000.0    0.0646      1225.24       MORTGAGE    520000.0   
68816    16000.0    0.1131       350.36       MORTGAGE     72000.0   

      verification_status   issue_d loan_status pymnt_plan    dti  ...  \
0         Source Verified  Mar-2019    low_risk          n  27.24  ...   
1          

In [33]:
df.drop(['home_ownership', 'verification_status'], axis=1, inplace=True) #Dropping any extra data that doesn't have a numeric data

In [34]:
X = pd.get_dummies(X) # turns my categorical column (column of labels) into indicator columns (columns of 0s and 1s); removing the strings

In [35]:
# Create our target
y = df["loan_status"] # YOUR CODE HERE
y[:5]

0    low_risk
1    low_risk
2    low_risk
3    low_risk
4    low_risk
Name: loan_status, dtype: object

In [36]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,loan_status_low_risk,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.994958,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.070831,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,1.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,1.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,1.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [37]:
# Check the balance of our target values
# YOUR CODE HERE
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [38]:
# Split the X and y into X_train, X_test, y_train, y_test
# YOUR CODE HERE
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [39]:
# Create the StandardScaler instance
# YOUR CODE HERE
from sklearn.preprocessing import StandardScaler 

scaler = StandardScaler()
# StandardScaler(X_train, X_test)

In [40]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
# YOUR CODE HERE
X_scaler = scaler.fit(X_train)

In [41]:
# Scale the training and testing data
# YOUR CODE HERE

X_tr_scaled = X_scaler.transform(X_train)
X_te_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [42]:
# Resample the training data with the BalancedRandomForestClassifier
# YOUR CODE HERE
from imblearn.ensemble import BalancedRandomForestClassifier

brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf.fit(X_tr_scaled, y_train)


BalancedRandomForestClassifier(random_state=1)

In [43]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
y_pred = brf.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5

In [44]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, y_pred)

array([[    0,   101],
       [    0, 17104]])

In [45]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.00      0.00      1.00      0.00      0.00      0.00       101
   low_risk       0.99      1.00      0.00      1.00      0.00      0.00     17104

avg / total       0.99      0.99      0.01      0.99      0.00      0.00     17205



In [48]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE
importances = brf.feature_importances_
index = np.argsort(importances)[::-1]
for f in range(X.shape[1]):
    print(f"{X.columns.values[index[f]]}: ({importances[index[f]]})")

loan_status_low_risk: (0.375986243493301)
loan_status_high_risk: (0.2560434813108393)
total_pymnt: (0.03801684877600174)
total_rec_int: (0.0356938863720125)
total_pymnt_inv: (0.02965387817725142)
total_rec_prncp: (0.02654691576206156)
last_pymnt_amnt: (0.026022594504139834)
issue_d_Jan-2019: (0.01030435416656268)
int_rate: (0.008695722881838274)
max_bal_bc: (0.006398159279402005)
tot_cur_bal: (0.005897864016971733)
tot_hi_cred_lim: (0.0058722491599039785)
installment: (0.005630249740396605)
mths_since_rcnt_il: (0.005310855730142811)
dti: (0.005283332383452439)
revol_bal: (0.0051949368616207746)
out_prncp: (0.005126285200423285)
avg_cur_bal: (0.005077792669655928)
il_util: (0.005029897475186519)
mths_since_recent_inq: (0.004809771670528303)
issue_d_Mar-2019: (0.004782106966849444)
annual_inc: (0.0044398851790157295)
total_bal_il: (0.0042982782736184335)
out_prncp_inv: (0.004178889346924406)
total_bc_limit: (0.003970060228376144)
loan_amnt: (0.0037637142767362685)
bc_util: (0.00375038962

### Easy Ensemble Classifier

In [50]:
# Train the Classifier
# YOUR CODE HERE
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(n_estimators = 100, random_state =1)
eec.fit(X_tr_scaled, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [54]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
from sklearn.metrics import balanced_accuracy_score

y_pred = eec.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5

In [55]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, y_pred)

array([[    0,   101],
       [    0, 17104]])

In [56]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.00      0.00      1.00      0.00      0.00      0.00       101
   low_risk       0.99      1.00      0.00      1.00      0.00      0.00     17104

avg / total       0.99      0.99      0.01      0.99      0.00      0.00     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.

4. What are the top three features?

    YOUR ANSWER HERE.

In [ ]:
# 1. Both accuracy score were very similar. They both were composed of a 0.00 high risk which was a bit trouble some, considering we are looking for high risk scores. In additon to both scores for low risk was at 0.99.
# 2. Both of the scores had the same recall scores as well of a 0.00, therefore I am unable to choose.
# 3. The scores were both 0.00.
# 4. The top three features were:
            #loan_status_low_risk: (0.375986243493301)
            #loan_status_high_risk: (0.2560434813108393)
            #total_pymnt: (0.03801684877600174)